# Model Experiments Phase - TSA 1001

#### Import Necessary Libraries

In [1]:
# Basic Libraies
import numpy as np
import pandas as pd
import seaborn as sns

import matplotlib.pyplot as plt
import warnings as wr
wr.filterwarnings('ignore')

In [3]:
#!pip install mlflow

In [4]:
# !pip install sktime

#### About Data

1. volume- v4 (best selected features with Ridge & Decesion Tree)- As per session
2. volumn-v5 (best selected features with Ridge & Ridge)

In [2]:
data = pd.read_parquet("D:/Study/PROJECT-REALTIME/PT-1001/model_output_data/output/sales_bya_v4.parquet") # I guest data not be requried, need confirmation
df = data.copy()

In [3]:
df.head()

,net_price_lag_14,net_price_window_7_std,date_quarter_end,net_price_lag_7,date_year_start,net_price_lag_3,net_price_lag_28,date_month_start,date_year_end,date_quarter_start,date_month_end,date_weekend,net_price,date
0,5.255228e+06,5.136896e+06,0,1.914946e+05,0,2599070.50,26272324.0,0,0,0,0,0,5042942.0,2019-08-08
1,1.155514e+07,5.042066e+06,0,1.105840e+07,0,4009107.25,15105817.0,0,0,0,0,1,2932270.5,2019-08-10
2,7.580881e+04,4.480915e+06,0,5.000808e+06,0,4451132.00,5594980.5,0,0,0,0,0,1809937.5,2019-08-12
3,3.132104e+06,4.958484e+06,0,2.071218e+04,0,5042942.00,9242950.0,0,0,0,0,0,3067519.0,2019-08-13
4,4.743207e+06,5.091923e+06,0,2.599070e+06,0,2932270.50,9014580.0,0,0,0,0,0,5123492.0,2019-08-14


In [4]:
df.shape

(1121, 14)

#### Loading Folds

In [5]:
folds = np.load("D:/Study/PROJECT-REALTIME/PT-1001/model_output_data/output/folds_v2.npz")
folds.files

['train_0', 'test_0', 'train_1', 'test_1', 'train_2', 'test_2']

In [6]:
# Access Folds
train = df.loc[folds['train_2']]
test = df.loc[folds['test_2']]

In [7]:
train.shape

(841, 14)

In [8]:
test.shape

(280, 14)

In [9]:
841+280 # verified

1121

In [10]:
841/1121

0.7502230151650312

In [11]:
# Training Set
X = train.drop(columns=['date','net_price'])
y = train['net_price']

In [12]:
# Test Set
Xt = test.drop(columns=['date','net_price'])
yt = test['net_price']

- We will train on train data & test on test set(280 data points)
- Still we are doing this on a single folder i.g (train_2, test_2)

In [13]:
# Check the whether the lenge of data is same for both train & test set
print(X.shape)
print(y.shape)
print(Xt.shape)
print(yt.shape)

(841, 12)
(841,)
(280, 12)
(280,)


#### Required Libraries with Mlflow ( Prerequisit for Modeling)

In [30]:
import mlflow
from sklearn.neighbors import KNeighborsRegressor                    # required for model i.g 3.1
from sklearn.metrics import r2_score, mean_absolute_percentage_error  # required for evaluation i.g 3.1
# mlflow.create_experiment('Net Price Forecasting')    # this is one time, return a unique ID / hash ID

<center><h1> Experiments </center></h1>

##### Test Model 1: KNeighbors TimeSeries Regressor

In [17]:
'''# import library
from sktime.regression.distance_based import KNeighborsTimeSeriesRegressor

# modeling
knn = KNeighborsTimeSeriesRegressor(
    n_neighbors=5, weights='uniform', algorithm='brute',
    distance='dtw', distance_params=None, 

"# import library\nfrom sktime.regression.distance_based import KNeighborsTimeSeriesRegressor\n\n# modeling\nknn = KNeighborsTimeSeriesRegressor(\n    n_neighbors=5, weights='uniform', algorithm='brute',\n    distance='dtw', distance_params=None,\n    distance_mtype=None, leaf_size=30, n_jobs=None)\n\n# fit the model\nknn.fit(X,y)"

##### Test Model 2: Ridge Regression

In [19]:
# import library
from sklearn.linear_model import Ridge

# Modeling 
model= Ridge(
    alpha=1.0, fit_intercept=True, copy_X=True, max_iter=None,
    tol=0.0001, solver='auto', positive=False, random_state=42,
)

# Fit the model
model.fit(X,y)

Ridge(random_state=42)

##### Test Model 3: KNeighbors Regressor - without Mlflow

In [14]:
# import library
from sklearn.neighbors import KNeighborsRegressor

# Modeling 
model= KNeighborsRegressor(n_neighbors=5, weights='uniform', metric='euclidean')

# Fit the model
model.fit(X,y)

KNeighborsRegressor(metric='euclidean')

#### Checking Shape- Error Fixation

In [20]:
Xt.shape

(280, 12)

In [21]:
yt.shape # not matched

(280,)

##### Test Model 3.1: KNeighbors Regressor  with mflow

In [18]:

mlflow.set_experiment('Net Price Forecasting')      # it will log hash executed for 'Net Price forecasting'
with mlflow.start_run():                            # it will create a session with mlflow
    model= KNeighborsRegressor(n_neighbors=7, weights='distance', metric='euclidean')
    model.fit(X,y)                                 # train the model on train data
    
    ### log params
    mlflow.log_params(dict(n_neighbors=7, weights='distance', metric='euclidean'))
    
    ### log metrics
    mlflow.log_metrics(
        {
            'r2': r2_score(model.predict(Xt), yt),  # it will cal r2_score on test data, Xt:prediction, yt: actual
            'mse': mean_absolute_percentage_error(model.predict(Xt), yt)
        }
    )
    
    ### log model   



###
# it will log params, metrics, model for every iteration / run

##### Test Model 4.1: Ridge with Mlflow

In [21]:
from sklearn.linear_model import Ridge

In [20]:
mlflow.create_experiment('Net Price Forecasting_Ridge_4.1')

'433896854597316750'

In [24]:

mlflow.set_experiment('Net Price Forecasting_Ridge_4.1')      
with mlflow.start_run(): 
    model2 = Ridge(
    alpha=1.0, fit_intercept=True, copy_X=True, max_iter=1000,
    tol=0.0001, solver='auto', positive=False, random_state=42,
    )
    
    model2.fit(X,y)                                 # train the model on train data
    
    ### log params
    mlflow.log_params(dict(alpha=1.0, fit_intercept=True, copy_X=True, max_iter=1000,
    tol=0.0001, solver='auto', positive=False, random_state=42))
    
    ### log metrics
    mlflow.log_metrics(
        {
            'r2': r2_score(model2.predict(Xt), yt),  # it will cal r2_score on test data, Xt:prediction, yt: actual
            'mse': mean_absolute_percentage_error(model2.predict(Xt), yt)
        }
    )
    
    ### log model 

## 5.0 Deicions Tree

#### 5.1 Decision Tree - Basic

In [36]:
# Evaluation metrics
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [58]:
# import library
from sklearn.tree import DecisionTreeRegressor

# Modeling
model = DecisionTreeRegressor(
    criterion='squared_error', splitter='best',
    max_depth=None, min_samples_split=2, min_samples_leaf=2,
    min_weight_fraction_leaf=0.0, max_features=None, random_state=None,
    max_leaf_nodes=None, min_impurity_decrease=0.0, ccp_alpha=0.0)

# fit  the model
model.fit(X, y)

DecisionTreeRegressor(min_samples_leaf=2)

In [27]:
# Prediction
model.predict(Xt)

array([7.12844375e+06, 4.47389675e+06, 1.16659205e+07, 2.54786706e+06,
       1.25810020e+07, 6.25024512e+06, 8.06907900e+06, 3.30533050e+06,
       3.25071638e+06, 3.29214375e+04, 9.37199950e+06, 4.11781212e+06,
       7.56782475e+06, 7.16074317e+06, 2.92954950e+06, 4.90515650e+06,
       5.26959250e+06, 4.28459925e+06, 9.48330382e+05, 5.96160233e+06,
       6.35626667e+06, 1.61682410e+07, 7.14422675e+06, 1.20859907e+06,
       3.07788075e+06, 9.07934900e+06, 5.72585850e+06, 5.11783650e+06,
       8.06907900e+06, 4.23794708e+06, 1.16659205e+07, 5.33616567e+06,
       5.77189925e+06, 5.55189375e+06, 8.06907900e+06, 3.25071638e+06,
       5.86183325e+06, 3.92281350e+06, 5.88077667e+06, 3.79034775e+06,
       3.92281350e+06, 7.79414383e+06, 7.79414383e+06, 3.14692600e+06,
       3.92281350e+06, 7.42620417e+06, 4.55669283e+06, 3.16227638e+06,
       6.35626667e+06, 2.21833920e+07, 4.93383175e+06, 5.49684050e+06,
       3.46274175e+06, 3.07788075e+06, 5.96261400e+06, 3.26529500e+06,
      

In [57]:
# Evaluation # Xt:actual, yt:prediction
print(f'MAPE :', mean_absolute_percentage_error(model.predict(Xt), yt))
print(f'R2 :', r2_score(model.predict(Xt), yt))
print(f'MSE :', mean_squared_error(model.predict(Xt), yt))
print(f'MAE :', mean_absolute_error(model.predict(Xt), yt))

MAPE : 12.006783510238181
R2 : -1.350986215972751
MSE : 34544224793325.707
MAE : 4215118.526584008


#### 5.2 : Decision Tree- v2

In [48]:
from sklearn.metrics import mean_absolute_percentage_error, r2_score, mean_squared_error, mean_absolute_error

In [66]:
# import
from sklearn.tree import DecisionTreeRegressor

# modeling
model_3 = DecisionTreeRegressor(
    criterion='poisson',
    splitter='random',
    max_depth=5,
    min_samples_split=3,
    min_samples_leaf=2,
    min_weight_fraction_leaf=0.0,
    max_features='auto',
    random_state=41,
    max_leaf_nodes=3,
    min_impurity_decrease=0.0,
    ccp_alpha=0.0,
)

# fit the model
model_3.fit(X, y)


DecisionTreeRegressor(criterion='poisson', max_depth=5, max_features='auto',
                      max_leaf_nodes=3, min_samples_leaf=2, min_samples_split=3,
                      random_state=41, splitter='random')

In [71]:
# Evaluation # Xt:actual, yt:prediction
print(f'MAPE :', mean_absolute_percentage_error(model_3.predict(Xt), yt))
print(f'R2 :', r2_score(model_3.predict(Xt), yt))
print(f'MSE :', mean_squared_error(model_3.predict(Xt), yt))
print(f'MAE :', mean_absolute_error(model_3.predict(Xt), yt))


MAPE : 0.7751645481899124
R2 : -11.151638054690437
MSE : 25624262415428.688
MAE : 3441879.2101823553


### 6.0 Random Forest

#### 6.1 Random Forest-Basic

In [77]:
from sklearn.ensemble import RandomForestRegressor

# modeling
model_4 = RandomForestRegressor(
    criterion='squared_error',
    max_depth=5, 
    min_samples_split=2,
    min_samples_leaf=1,
    min_weight_fraction_leaf=0.0,
    max_features=1.0,
    max_leaf_nodes=None,
    min_impurity_decrease=0.0,
    bootstrap=True,
    oob_score=False,
    n_jobs=None,
    random_state=33,
    verbose=0,
    warm_start=False,
    ccp_alpha=0.0,
    max_samples=None,
)

# train the model
model_4.fit(X, y)

RandomForestRegressor(max_depth=5, random_state=33)

In [78]:
# Prediction & Evaluation
print(f'MAPE :', mean_absolute_percentage_error(model_4.predict(Xt), yt))
print(f'R2 :', r2_score(model_4.predict(Xt), yt))
print(f'MSE :', mean_squared_error(model_4.predict(Xt),yt))
print(f'MAE :', mean_absolute_error(model_4.predict(Xt), yt))

MAPE : 0.6616110862986831
R2 : -4.439369404314615
MSE : 23657300931616.277
MAE : 3338817.996242817


### 7.0 XGBoost

#### 7.1  XGBoost

In [83]:
# !pip3 install xgboost

In [86]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error as MSE 

# Instantiation 
xgb_r = xgb.XGBRegressor(objective ='reg:linear', 
                  n_estimators = 10, seed = 123) 

# Fitting the model 
xgb_r.fit(X,y)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=10, n_jobs=None,
             num_parallel_tree=None, objective='reg:linear', ...)

In [89]:
# Predict the model 
# xgb_r.predict(Xt) 

In [87]:
print(f'MAPE :', mean_absolute_percentage_error(xgb_r.predict(Xt), yt))
print(f'R2 :', r2_score(xgb_r.predict(Xt), yt))
print(f'MSE :', mean_squared_error(xgb_r.predict(Xt),yt))
print(f'MAE :', mean_absolute_error(xgb_r.predict(Xt), yt))

MAPE : 0.67495173
R2 : -4.119310378022047
MSE : 23520476000000.0
MAE : 3345206.8


####  RMSE Computation 

In [90]:
pred = xgb_r.predict(Xt)

In [91]:
rmse = np.sqrt(MSE(yt, pred)) 
print("RMSE : % f" %(rmse)) 

RMSE :  4849791.500000


### Task (Takeaway)

1. Task1: Import libraries
2. Task2: load best selected features, not data
3. Task3: Experiments
4. Task4: Modeling i.g KNeighbors

### Splitting

In [24]:
# ts = TimeSeriesSplit(n_splits=2, max_train_size=800, test_size=25, gap=0) # since max data is 1100+ so, max_train_size 800, gap = no gap in spliting in date range
# folds = list(ts.split(df))                                                              # the generator has 2 folds that we can asscess, if you c

In [ ]:
# Accessing 
# list(ts.split(df)) [0] # train index

In [ ]:
# list(ts.split(df)) [1] # test index

In [ ]:
# storing folds
train = df.loc[folds[1][0]] # training 2 fold, 1st index for train
test = df.loc[folds[1][1]]  # testing 2 fold, 2nd index for test

In [ ]:
train.shape 

In [ ]:
test.shape

## 3.0 Experiments- Testing

#### code Narraation - Splitter

1. import package i.g TimeSeriesSplit(n_splits=2, max_train_size=800, test_size=25, gap=0) 
2. ts.split(df) > it will return a generator from that we can access, but for access need to convert into list first
3. list(ts.split(df)) > converted to get get & access the folds/ data. Check the index
4. store this index somewhere i.g folds  example folds = list(ts.split(df))
5. access folds
- train = df.loc[folds[1]]  >> two folds 0, 1 >> we are targeting 2nd or the folds with max data i.g 1 and then take only few not all, that is training i.g 0 as below:
- train = df.loc[folds[1][0]]

<center> <h3> End of model (model1) Testing </h3></center>

#### Attempts

1. KNeighborsTimeSeriesRegressor  -- Failed >>  hten approach to Darts model with XGBoost Model (

##### Genearal Notes

1. We will take best features and do modeling on that to see which one is performing well
2. No need to take data in this part
3. We need the terminal > jupyer > new > terminal

#### Darts Model

1. Not by default need to install
2. Darts expect params to pass, it generate differnet time series (requirment e.g lags, window etc) automatically, but we already convert time series data to tabular form
3. So, need to go with normal machine learing approach l.g sklearn
4. We haave already calculated lags, window, expanding window, so the task of dart we already completed!


In [ ]:
# Modeling Steps
'''
1. Import necessary library
2. Train & Test dataset (train test or load train / test data)
3. mode architecture
4. result interpretion/analysis

'''

#### Ref

1. https://tslearn.readthedocs.io/en/latest/gen_modules/neighbors/tslearn.neighbors.KNeighborsTimeSeriesRegressor.h